In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
path = "/content/drive/My Drive/Khoa Luan/"

In [3]:
!pip install -q keras-bert==0.85.0      #cài thư  viện keras-bert
!pip install -q keras-rectified-adam      #cài thư viện RAdam

In [4]:
#@title xài tensorflow 1.x thì nó mới chạy đc
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
#@title xài keras 2.3.1
!pip install keras==2.3.1

     |████████████████████████████████| 378kB 9.2MB/s 
ERROR: keras-bert 0.85.0 has requirement Keras>=2.4.3, but you'll have keras 2.3.1 which is incompatible.
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [6]:
# import các thư viện
import codecs
import tensorflow as tf
import keras
import os
from keras_radam import RAdam
from keras import backend as K
from keras_bert import load_trained_model_from_checkpoint
import numpy as np

Using TensorFlow backend.


In [7]:
!wget -q https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip  #tải pre-train BERT
!unzip -o multi_cased_L-12_H-768_A-12.zip #giải nén pre-train BERT

Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  


In [8]:
pretrained_path = 'multi_cased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [9]:
SEQ_LEN = 256 #sequence_lenght là 256

In [10]:
#load model BERT lên
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    seq_len=SEQ_LEN,
    output_layer_num=4,
    trainable=True
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
import keras_bert
from keras.layers import Dense, Input, Flatten,SpatialDropout1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout,GlobalMaxPool1D,Lambda
from keras.models import Model
from keras.layers import Bidirectional,LSTM,GRU
from keras.optimizers import Adam

In [12]:
LR=2e-5

In [13]:
class NonMasking(keras.layers.Layer):    #hàm này dùng để non mask output của BERT (dùng cho các mô hình kết hợp của BERT)
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape  

In [14]:
def bert_rcnn():   #hàm xây dựng BERT-RCNN
  inputs = model.inputs
  bert_out = NonMasking()(model.outputs)

  bert_out = SpatialDropout1D(0.2)(bert_out)

  l_embedding = Lambda(lambda x: K.concatenate([K.zeros(shape=(K.shape(x)[0], 1, K.shape(x)[-1])),
                                                        x[:, :-1]], axis=1))(bert_out)
          
  r_embedding = Lambda(lambda x: K.concatenate([K.zeros(shape=(K.shape(x)[0], 1, K.shape(x)[-1])),
                                                        x[:, 1:]], axis=1))(bert_out)

  forward = LSTM(256, return_sequences=True)(l_embedding) 
  backward = LSTM(256, return_sequences=True, go_backwards=True)(r_embedding)
  backward = Lambda(lambda x: K.reverse(x, axes=1))(backward)

  together = [forward, bert_out , backward]

  together = Concatenate(axis=2)(together)

  semantic = Conv1D(256, kernel_size=1, activation="relu")(together)
  sentence_embed = Lambda(lambda x: K.max(x, axis=1))(semantic)

  dense_layer = Dense(256, activation='relu')(sentence_embed)
  preds = Dense(1, activation='sigmoid')(dense_layer)
  model1 = Model(inputs, preds)
  return model1

In [15]:
model_rcnn = bert_rcnn()
model_rcnn.compile(loss='binary_crossentropy',    #compile mô hình BERT-RCNN
              optimizer=RAdam(learning_rate=LR),
              metrics=['acc'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
model_rcnn.load_weights(path+'256_checkpoint_rcnn2_data1.h5') #load file weights BERT-RCNN lên

In [ ]:
from keras_bert.layers import Extract
def bert_base():              #xây dựng mô hình BERT-base
  inputs = model.inputs[:2]               
  newout = Extract(index=0)(model.output)
  newout = keras.layers.Dense(256,activation='relu')(newout)
  outputs = keras.layers.Dense(units=1, activation='sigmoid')(newout)
  model1 = keras.models.Model(inputs, outputs)
  return model1

In [ ]:
model_base = bert_base()
model_base.compile(           #compile mô hình BERT-base
    Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'],
  )

In [ ]:
model_base.load_weights(path+'256_checkpoint_data1.h5')   #load file Weight mô hình BERT-base lên

In [17]:
token_dict = {}    
with codecs.open(vocab_path, 'rb','utf-8') as reader: #load file vocab vào token_dict
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict) 

In [ ]:
vua ve lai vn la phai lap tai khoan cua hang ee ord do cua me yumi ngay chua bao gio that vong negative negative yeu thuong positive





In [18]:
from keras_bert import Tokenizer
index = Tokenizer(token_dict,cased=True)

In [19]:
#@title cài thư viện vncorenlp
!pip3 install vncorenlp
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     |████████████████████████████████| 2.7MB 8.6MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645935 sha256=fb7e5532e49fa54ad8342bb1eadc1131b344a39feaacea2ba912d942fd1bdca4
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp
--2020-07-31 07:50:58--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   113MB/s    in 0.2s    

2020-07-31 07:50:59 (113 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2020-07-31 07:51:01--  https

In [20]:
from vncorenlp import VnCoreNLP
tokenizer = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [21]:
import pickle 
replace_list = pickle.load(open(path+'replace_list.pkl','rb')) #load file để chỉnh từ viết tắt, emo.

In [22]:
import re
from gensim.utils import simple_preprocess
import pandas as pd
from nltk import flatten

In [23]:
with open(path+'pos.txt','r') as f:   #load file từ vựng tích cực
    Pos_list = [i.strip() for i in f]

In [24]:
with open(path+'neg.txt', 'r') as f: #load file từ vựng tiêu cực
    Neg_list = [i.strip() for i in f]

In [25]:
def preprocess(text):   
  check = re.search(r'([a-z])\1+',text)
  if check:
    if len(check.group())>2:
      text = re.sub(r'([a-z])\1+', lambda m: m.group(1), text, flags=re.IGNORECASE) #remove các ký tự kéo dài như hayyy,ngonnnn...

  text = text.strip() #loại dấu cách đầu câu
  text = text.lower() #chuyển tất cả thành chữ thường

  text = re.sub('< a class.+</a>',' ',text)

  for k, v in replace_list.items():       #replace các từ có trong replace_list
    text = text.replace(k, v)       

  text = re.sub(r'_',' ',text)  
  
  text = ' '.join(i for i in flatten(tokenizer.tokenize(text)))             #gán từ ghép
  
  for i in Pos_list:                                    #thêm feature positive
    if re.search(' '+i+' ',text): 
      text = re.sub(i,i+' positive ',text)
  for i in Neg_list:                                    #thêm feature negative
    if re.search(' '+i+' ',text):
      text = re.sub(i,i+' negative ',text)
  return text

In [26]:
def load_data(text):        #đưa các từ thành index
    global tokenizer
    indices = []
    ids, segments = index.encode(text, max_len=SEQ_LEN)
    indices.append(ids)

    return [indices, np.zeros_like(indices)]

In [27]:
def predict_text(text,model_selection='base'):  #hàm predict câu đầu vào, model_selection để chọn mô hình, nếu là 'base' thì dùng BERT-base, rcnn thì dùng BERT-RCNN
  text = preprocess(text)           #preprocess câu đầu vào
  text_input = load_data(text)      #chuyển thành index
  if model_selection=='rcnn':                   
    print(np.round(model_rcnn.predict(text_input)))   #predict câu bằng BERT-RCNN
  elif model_selection=='base':
    print(np.round(model_base.predict(text_input)))   #predict câu bằng BERT-base

In [42]:
text = "sau khi an thu hat thi co ly do khong nen mua thu qua thi bi negative diu iu thu qua qua hoi negative dau thu qua het qua nho negative thu qua co qua bi negative moc negative den thui cac ban tinh ki khi mua nha re positive hon vai chuc nhung mat nua hon tien do"
predict_text(text,model_selection='rcnn')

[[0.]]
